In [ ]:
! pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
spark = SparkSession.builder.appName("my_app").getOrCreate()
schema_car = StructType([StructField("year", StringType(), True)
 ,StructField("make", StringType(), True)
 ,StructField("model", StringType(), True)
 ,StructField("trim", StringType(), True)
 ,StructField("body", StringType(), True)
 ,StructField("transmission", StringType(), True)
 ,StructField("vin", StringType(), True)
 ,StructField("state", StringType(), True)
 ,StructField("condition", IntegerType(), True)
 ,StructField("odometer", IntegerType(), True)
 ,StructField("color", StringType(), True)
 ,StructField("interior", StringType(), True)
 ,StructField("seller", StringType(), True)
 ,StructField("mmr", IntegerType(), True)
 ,StructField("sellingprice", IntegerType(), True)
 ,StructField("saledate", StringType(), True)
])

In [ ]:
spark_cardataframe = spark.read.option("header", True).csv(r'/content/car_prices.csv',schema=schema_car)

In [ ]:
spark_cardataframe.printSchema()

In [ ]:
spark_cardataframe.show(n=2,vertical=True,truncate=150)

In [ ]:
spark_cardataframe.createOrReplaceTempView("car_sales_test")

In [ ]:
def execsql(query):
    return(spark.sql(query))

select_count = 'select count(*) from car_sales_test;'

In [ ]:
from os import execlpe
execsql(select_count).show()

In [ ]:
groupby_col = 'select year, count(*) as vehicle_count from car_sales_test group by year order by count(*) desc;'
execsql(groupby_col).show(10)

# Inference Year 2012 has most sales

In [ ]:
groupby_col = 'select make, count(*) as vehicle_count from car_sales_test group by make order by count(*) desc;'
execsql(groupby_col).show(10)

# Highest Sales company wise

In [ ]:
groupby_col = 'select make,model,count(*) as vehicle_count from car_sales_test group by make,model order by count(*) desc;'
execsql(groupby_col).show(10)

# Highest Sales on make / Model

In [ ]:
groupby_col = 'select make,model,count(*),year as vehicle_count from car_sales_test group by make,model,year order by count(*) desc;'
execsql(groupby_col).show(10)

# Highest Sales on make / Model / Year

In [ ]:
groupby_col = 'select model,year,count(*) as vehicle_count from car_sales_test group by model,year order by count(*) desc;'
execsql(groupby_col).show(10)

In [ ]:
groupby_col = 'select make,year,count(*) as vehicle_count from car_sales_test group by make,year order by count(*) desc;'
execsql(groupby_col).show(10)

In [ ]:
query = 'select make,model,count(*) from car_sales_test group by 2,1 order by 3 desc'
execsql(query).show(5)

In [ ]:
query = '''select make , model , count(*) as vcount, min(odometer) , max(odometer) as maxo , round(avg(odometer),1) as avgmiles
from car_sales_test
where odometer <> 999999
and odometer <> 1
and make <> 'NULL'
group by make, model order by 3 desc
'''
execsql(query).show()

In [ ]:
query = ''' select model,state,count(distinct seller) as dealer_count
 from car_sales_test
 where odometer <> 999999
and odometer <> 1
and make <> 'NULL'
 group by model,state
 order by dealer_count desc
'''
execsql(query).show()

In [ ]:
query = ''' select state,count(distinct seller) as dealer_count
 from car_sales_test
 where odometer <> 999999
and odometer <> 1
and make <> 'NULL'
 group by state
 order by dealer_count desc
'''
execsql(query).show()

In [ ]:
query = ''' select state, count(distinct seller) filter (where make in ('Nissan')) as dealer_count
 from car_sales_test
 group by state
'''
execsql(query).show()

In [ ]:
query = ''' select state,make,model,year, count(*)
filter ( where make in ('Nissan','Ford','GM')) as vechicle_count
from car_sales_test
 group by grouping sets((state),(state,make),(state,make,model),(state,make,model,year))
 having vechicle_count <> 0
 order by   vechicle_count  desc
'''
execsql(query).show(50)

In [ ]:
query = ''' select state,make, count(*)
filter ( where make in ('Nissan','Ford','GM') and state in ('fl','tx') ) as vechicle_count
from car_sales_test
 group by grouping sets((state),(state,make))
 having vechicle_count <> 0
 order by   vechicle_count  desc
'''
execsql(query).show(50)

In [ ]:
query = ''' select state,make, count(*)
filter ( where make in ('Nissan','Ford','GM') and state in ('fl','tx') ) as vechicle_count
from car_sales_test
 group by state,make
 with rollup
 having vechicle_count <> 0
 order by   vechicle_count  desc
'''
execsql(query).show(50)

# Interesting logic - admiring it

In [ ]:
query = ''' select state,make, count(*)
filter ( where make in ('Nissan','Ford','GM') and state in ('fl','tx') ) as vechicle_count
from car_sales_test
 group by state,make
 with cube
 having vechicle_count <> 0
 order by   vechicle_count  desc
'''
execsql(query).show(50)

# Interesting logic - admiring it